In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from cicada.communicator import SocketCommunicator
import cicada.logging

def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Informational message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Informational message from player 0!
INFO:root:Informational message from player 1!


In [3]:
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    log.warning(f"Warning message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

In [4]:
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    log.error(f"Error message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

ERROR:root:Error message from player 0!
ERROR:root:Error message from player 1!


In [5]:
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    log.critical(f"Critical message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

CRITICAL:root:Critical message from player 0!
CRITICAL:root:Critical message from player 1!


In [6]:
logging.getLogger().setLevel(logging.DEBUG) # Don't filter DEBUG messages

def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    log.debug(f"Debug message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

DEBUG:root:Debug message from player 0!
DEBUG:root:Debug message from player 1!


In [7]:
logging.getLogger().setLevel(logging.INFO) # Set the log level back to normal

In [8]:
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello only from player {communicator.rank}!", src=1)

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello only from player 1!
